In [1]:
from glob import glob
from os.path import join
import pandas as pd
import numpy as np

from load_results_lib import load_moment_df, add_metadata
from load_results_lib import add_deviation_stats, add_derived_stats
from load_results_lib import VALID_METHODS

In [2]:
base_folder = '../blade_runs/'

folder_method_list = (
    (join(base_folder, "nuts_results/"), 'NUTS'),
    (join(base_folder, "dadvi_results/"), 'DADVI'),
    (join(base_folder, "lrvb_results/"), 'LRVB'),
    (join(base_folder, "raabbvi_results/"), 'RAABBVI'),
    (join(base_folder, "sadvi_results/"), 'SADVI'),
    (join(base_folder, "sfullrank_advi_results/"), 'SADVI_FR'),
    (join(base_folder, 'lrvb_doubling_results'), 'LRVB_Doubling')
)

all_results = dict()

for cur_folder, cur_method in folder_method_list:

    print(cur_method, cur_folder)

    data = load_moment_df(cur_folder)

    data = add_metadata(data, cur_method)

    all_results[cur_method] = data

print('Done!')

NUTS ../blade_runs/nuts_results/
DADVI ../blade_runs/dadvi_results/


LRVB ../blade_runs/lrvb_results/
RAABBVI ../blade_runs/raabbvi_results/
SADVI ../blade_runs/sadvi_results/
SADVI_FR ../blade_runs/sfullrank_advi_results/
LRVB_Doubling ../blade_runs/lrvb_doubling_results
Done!


In [13]:
raabbvi_maxiter = 19900

method_1 = 'LRVB_Doubling'
method_2 = 'RAABBVI'

#method_1_df = add_deviation_stats(all_results[method_1], all_results['NUTS']).dropna()
method_1_df = add_deviation_stats(all_results[method_1], all_results['NUTS'])

print(method_1_df)

                                       draw_dict_path  \
0   ../blade_runs/lrvb_doubling_results/draw_dicts...   
1   ../blade_runs/lrvb_doubling_results/draw_dicts...   
2   ../blade_runs/lrvb_doubling_results/draw_dicts...   
3   ../blade_runs/lrvb_doubling_results/draw_dicts...   
4   ../blade_runs/lrvb_doubling_results/draw_dicts...   
..                                                ...   
61  ../blade_runs/lrvb_doubling_results/draw_dicts...   
62  ../blade_runs/lrvb_doubling_results/draw_dicts...   
63  ../blade_runs/lrvb_doubling_results/draw_dicts...   
64  ../blade_runs/lrvb_doubling_results/draw_dicts...   
65  ../blade_runs/lrvb_doubling_results/draw_dicts...   

                                                means  \
0   {'Intercept': 97.86866517275877, 'treatment': ...   
1   {'Intercept': 0.3586016277938707, 'c_dist100':...   
2   {'Intercept': -0.9352684764451462, 'x': [15.28...   
3   {'Intercept': 97.07068567951451, 'treatment': ...   
4   {'Intercept': 9.7451798181

/home/rgiordan/Documents/git_repos/DADVI/dadvi-experiments/comparison/analysis/load_results_lib.py:30: RuntimeWarning: invalid value encountered in divide
  return {x: (mean_dict[x] - ref_mean_dict[x]) / ref_sd_dict[x] for x in mean_dict}
/home/rgiordan/Documents/git_repos/DADVI/dadvi-experiments/comparison/analysis/load_results_lib.py:34: RuntimeWarning: invalid value encountered in divide
  return {x: (sd_dict[x] - ref_sd_dict[x]) / ref_sd_dict[x] for x in ref_sd_dict}


In [14]:
method_1_df['means'][0]

{'Intercept': 97.86866517275877,
 'treatment': array([4.44689546]),
 'pre_test': array([0.46459963]),
 'post_test_sigma': 8.224619901361285}

In [1]:
method_1_df = add_derived_stats(method_1_df)

method_2_df = add_deviation_stats(all_results[method_2], all_results['NUTS']).dropna()
method_2_df = add_derived_stats(method_2_df)

NameError: name 'add_deviation_stats' is not defined

In [ ]:
print(all_results['SADVI']['converged'].mean())
print(all_results['RAABBVI']['converged'].mean())

In [ ]:
all_results['SADVI']

In [ ]:
comparison = method_1_df.merge(method_2_df, on='model_name', 
                                         suffixes=(f'_{method_1}', f'_{method_2}'))

comparison

In [ ]:
#raabvi_with_deviations.iloc[0]['metadata'].keys()

In [ ]:
import matplotlib.pyplot as plt

f, ax = plt.subplots(1, 1)

xmin, xmax = [comparison[f'mean_rms_{method_1}'].min(), comparison[f'mean_rms_{method_1}'].max()]
# ax.scatter(comparison['mean_rms_raabbvi'], comparison['mean_rms_lrvb'], c=comparison['converged'])
ax.scatter(comparison[f'mean_rms_{method_1}'], comparison[f'mean_rms_{method_2}'])
ax.plot([xmin, xmax], [xmin, xmax])

for row in comparison.itertuples():
    ax.annotate(row.model_name, (getattr(row, f'mean_rms_{method_1}'), getattr(row, f'mean_rms_{method_2}')))

ax.set_xscale('log')
ax.set_yscale('log')

ax.set_xlabel(f'RMSE mean scaled by posterior sd, {method_1}')
ax.set_ylabel(f'RMSE mean scaled by posterior sd, {method_2}')

ax.grid(alpha=0.5, linestyle='--')

f.set_size_inches(12, 8)
f.tight_layout()

# plt.savefig('./mean_comparison.png', dpi=300)


In [ ]:
f, ax = plt.subplots(1, 1)

xmin, xmax = [comparison[f'sd_rms_{method_1}'].min(), comparison[f'sd_rms_{method_1}'].max()]
# ax.scatter(comparison['mean_rms_raabbvi'], comparison['mean_rms_lrvb'], c=comparison['converged'])
ax.scatter(comparison[f'sd_rms_{method_1}'], comparison[f'sd_rms_{method_2}'])
ax.plot([xmin, xmax], [xmin, xmax])

for row in comparison.itertuples():
    ax.annotate(row.model_name, (getattr(row, f'sd_rms_{method_1}'), getattr(row, f'sd_rms_{method_2}')))

ax.set_xscale('log')
ax.set_yscale('log')

ax.set_xlabel(f'RMSE sd scaled by posterior sd, {method_1}')
ax.set_ylabel(f'RMSE sd scaled by posterior sd, {method_2}')

ax.grid(alpha=0.5, linestyle='--')

f.set_size_inches(12, 8)
f.tight_layout()

# plt.savefig('./sd_comparison.png', dpi=300)

In [ ]:

f, ax = plt.subplots(1, 1)

xmin, xmax = [comparison[f'runtime_{method_1}'].min(), comparison[f'runtime_{method_1}'].max()]
# ax.scatter(comparison['mean_rms_raabbvi'], comparison['mean_rms_lrvb'], c=comparison['converged'])
ax.scatter(comparison[f'runtime_{method_1}'], comparison[f'runtime_{method_2}'])
ax.plot([xmin, xmax], [xmin, xmax])

for row in comparison.itertuples():
    ax.annotate(row.model_name, (getattr(row, f'runtime_{method_1}'), getattr(row, f'runtime_{method_2}')))

ax.set_xscale('log')
ax.set_yscale('log')

ax.set_xlabel(f'Runtime, {method_1}')
ax.set_ylabel(f'Runtime, {method_2}')

ax.grid(alpha=0.5, linestyle='--')

f.set_size_inches(12, 8)
f.tight_layout()

# plt.savefig('runtime_comparison.png', dpi=300)

In [ ]:
comparison.head()

In [ ]:
all_results['LRVB_Doubling']['M'] = all_results['LRVB_Doubling']['metadata'].apply(lambda x: x['M'])

In [ ]:
all_results['LRVB_Doubling'][['model_name', 'runtime', 'M']].sort_values('M', ascending=False).head(20)

In [ ]:
all_results['LRVB'][['model_name', 'runtime']].sort_values('runtime')